In [1]:
# delete all folders in working directory, leave files alone
!find . -mindepth 1 -maxdepth 1 -type d -exec rm -r {} +

In [2]:
!pip install unsloth
!pip install instructor
!pip install openai 
!pip install pydantic
!pip install dotenv
!pip install huggingface_hub
!python -m pip install --upgrade typing_extensions
!pip install vllm
!pip install mistralai

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 317.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 147.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 184.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 111.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 162.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 197.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 256.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 237.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 187.8 MB/s eta 0:00:00

In [ ]:
import os
import re
import torch
from unsloth import FastLanguageModel
from vllm import SamplingParams
from huggingface_hub import HfApi
from openai import OpenAI
import json
import time
import random
import logging
from functools import wraps
import hashlib

THERAPIST_CONFIG = {
    "model_name": "Llama-3-3B-Instruct",
    "hf_base_model_repo": "meta-llama/Llama-3.2-3B-Instruct", 
    "output_directory": "simulation_outputs_llama3_3b_instruct",
    "retry_max_attempts": 10,
}


HF_AUTH_TOKEN = ""
OPENAI_API_KEY = "" 

THERAPIST_MAX_SEQ_LENGTH = 9000
OPENAI_PATIENT_MODEL_NAME = "gpt-5-mini"
NUM_TOTAL_SIMULATIONS = 150
NUM_DIALOGUE_TURNS_PER_SIMULATION = 25
RETRY_BASE_DELAY = 5
KEYWORD_THINKING = "Thinking:"
KEYWORD_ANSWER = "Answer:"
PATIENT_PROFILES_FILENAME = "standardized_patient_profiles.json" 


USE_PATIENT_RESPONSE_SUPERVISOR = False

os.makedirs(THERAPIST_CONFIG["output_directory"], exist_ok=True)
main_log_file_path = os.path.join(THERAPIST_CONFIG["output_directory"], "simulation_run.log")

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
file_handler = logging.FileHandler(main_log_file_path, mode='w')
file_handler.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

if not logger.hasHandlers():
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)

logger.info("Logging initialized for model: %s. All logs will be saved to %s", THERAPIST_CONFIG['model_name'], main_log_file_path)

assert OPENAI_API_KEY and not OPENAI_API_KEY.startswith("sk-your_") and OPENAI_API_KEY != "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx", "OpenAI API Key is not set. Please replace placeholder."
assert HF_AUTH_TOKEN != "YOUR_HUGGINGFACE_TOKEN_HERE", "Hugging Face Token is not set. Please replace placeholder."
logger.debug("API keys seem to be set (not placeholders).")

def retry_with_backoff(retries=THERAPIST_CONFIG["retry_max_attempts"], base_delay=RETRY_BASE_DELAY, allowed_exceptions=(Exception,)):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < retries:
                try:
                    return func(*args, **kwargs)
                except allowed_exceptions as e:
                    attempts += 1
                    if attempts >= retries:
                        logger.error(f"Function {func.__name__} failed after {retries} attempts: {e}", exc_info=True)
                        raise
                    wait_time = base_delay * (2 ** (attempts - 1)) + random.uniform(0, 1)
                    logger.warning(f"Attempt {attempts}/{retries} for {func.__name__} failed with {type(e).__name__}: {e}. Retrying in {wait_time:.2f}s...")
                    time.sleep(wait_time)
            return None
        return wrapper
    return decorator

def load_therapist_model(model_repo: str, model_nickname: str, max_seq_length: int, hf_token: str):
    logger.info(f"Loading base model '{model_nickname}' from Hugging Face repo: {model_repo}")
    assert hf_token, "Hugging Face token is required for downloading gated models like Llama."

    try:
        api = HfApi()
        api.repo_info(repo_id=model_repo, token=hf_token)
        logger.info(f"Successfully accessed repo info for {model_repo} using provided token.")
    except Exception as e:
        logger.error(f"Could not access Hugging Face repo {model_repo}. Ensure your token is valid and has accepted the model's terms.", exc_info=True)
        raise

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_repo,
        max_seq_length=max_seq_length,
        dtype=torch.float16,
        load_in_4bit=False,
        token=hf_token,
        fast_inference=True,
    )

    assert model is not None, f"Failed to load {model_nickname} model."
    assert tokenizer is not None, f"Failed to load {model_nickname} tokenizer."

    logger.info(f"{model_nickname} therapist model and tokenizer loaded successfully in 16-bit with vLLM backend.")
    return model, tokenizer

def get_openai_client(api_key: str):
    assert api_key, "OpenAI API Key is invalid or not set."
    logger.info("Initializing OpenAI client...")
    client = OpenAI(api_key=api_key)
    logger.info("OpenAI client initialized successfully.")
    return client

def generate_synthetic_patient_profile():
    archetypes = [
        {
            "name": "The Hopeless Skeptic",
            "interaction_style": "Argumentative/Resistant",
            "psych_mindedness": "Low",
            "style_description": "Challenges therapist's suggestions, expresses strong doubts, may focus on perceived flaws in ACT.",
            "persona_prompt_detail": "You are deeply skeptical that this therapy can help you. You frequently challenge the therapist's questions with 'How is that supposed to help?' or 'I've tried that, it doesn't work.'"
        },
        {
            "name": "The Intellectualizer",
            "interaction_style": "Intellectualizing",
            "psych_mindedness": "Moderate",
            "style_description": "Analyzes feelings rather than experiencing them, uses abstract language, may resist experiential exercises.",
            "persona_prompt_detail": "You have a habit of talking *about* your feelings instead of feeling them. You use big words and complex ideas to keep a safe distance from difficult emotions like sadness or fear. When the therapist asks a simple question about a feeling, you might respond with a theory or an analysis instead of a direct answer."
        },
        {
            "name": "The Anxious Fortune-Teller",
            "interaction_style": "Catastrophizing/Future-Focused",
            "psych_mindedness": "Moderate",
            "style_description": "Immediately jumps to the worst-case scenario in any situation. Speaks about future disasters as if they are certain facts. Often uses absolute language like 'always,' 'never,' or 'guaranteed.'",
            "persona_prompt_detail": "Your mind is a 'fortune-telling' machine that only predicts disaster. When you talk about a problem, you immediately describe the worst possible chain of events that will 'definitely' happen. You are completely hooked by these stories. If the therapist asks you to consider other outcomes, you dismiss them as unrealistic. Your anxiety is tied to these future predictions, not the present moment."
        },
        {
            "name": "The Overwhelmed Avoider",
            "interaction_style": "Vague/Defensive",
            "psych_mindedness": "Low",
            "style_description": "Responds with 'I don't know,' changes the subject when uncomfortable, gives short, vague answers.",
            "persona_prompt_detail": "You find talking about your problems intensely uncomfortable. When the therapist gets too close to a sensitive topic, your go-to responses are 'I don't know,' 'I guess,' or you might try to change the subject. You aren't trying to be difficult, you're just overwhelmed and avoiding the feeling."
        },
    ]
    chosen_archetype = random.choice(archetypes)
    archetype_name = chosen_archetype["name"]
    interaction_style = chosen_archetype["interaction_style"]
    psych_mindedness = chosen_archetype["psych_mindedness"]
    style_description = chosen_archetype["style_description"]
    persona_prompt_detail = chosen_archetype["persona_prompt_detail"]

    age_options = [
        (18, 24, "a young adult"), (25, 34, "an adult in their late twenties or early thirties"),
        (35, 49, "a middle-aged individual"), (50, 64, "an individual in their early fifties to mid-sixties"),
        (65, 79, "a senior individual"), (80, 99, "an elderly individual"),
    ]
    age = random.choice(age_options)
    age_range, age_description = f"{age[0]}-{age[1]}", age[2]
    gender = random.choice(["male", "female", "non-binary"])
    occupations = [
        "software developer", "teacher", "nurse", "artist", "accountant", "student", "manager",
        "construction worker", "chef", "social worker", "business owner", "unemployed", "data scientist"
    ]
    occupation = random.choice(occupations)
    mental_health_issues = [
        ("mild anxiety", "occasional panic attacks, general worry"),
        ("moderate depression", "low energy, difficulty concentrating, loss of interest"),
        ("generalized anxiety disorder", "persistent worry, restlessness, muscle tension"),
        ("social anxiety", "intense fear of social judgment, avoidance of social situations"),
        ("PTSD", "flashbacks, nightmares, hypervigilance related to past trauma"),
        ("OCD", "intrusive thoughts, compulsive behaviors (e.g., checking, cleaning)"),
        ("burnout", "emotional exhaustion, cynicism, reduced efficacy related to work/stress"),
        ("adjustment disorder", "difficulty coping with a specific stressor (e.g., move, job change)"),
        ("low self-esteem", "pervasive feelings of inadequacy, harsh self-criticism"),
        ("grief", "prolonged sadness, difficulty functioning after a significant loss")
    ]
    mental_health_issue, symptom_description = random.choice(mental_health_issues)
    life_events = [
        "a recent difficult breakup", "the loss of a loved one", "job loss or instability",
        "a recent move", "ongoing financial stress", "starting a demanding new job or school program",
        "significant family conflict", "a health scare", "feeling isolated or lonely", "major life transition (e.g., empty nest)"
    ]
    life_event = random.choice(life_events)
    personalities = [
        ("introverted", "analytical", "cautious"), ("extroverted", "expressive", "action-oriented"),
        ("reserved", "detail-oriented", "anxious"), ("outgoing", "adaptable", "sometimes impulsive"),
        ("calm", "thoughtful", "private"), ("sensitive", "creative", "prone to self-doubt"),
        ("pragmatic", "organized", "skeptical"), ("gregarious", "optimistic", "easily distracted")
    ]
    personality1, personality2, personality3 = random.choice(personalities)
    coping_mechanisms = [
        "talking to friends/family", "avoiding triggers", "engaging in hobbies", "exercise",
        "mindfulness/meditation", "overworking", "substance use (mild/moderate)", "seeking reassurance",
        "intellectualizing feelings", "emotional eating", "procrastination", "using humor/sarcasm"
    ]
    coping_mechanism = random.choice(coping_mechanisms)
    backgrounds = [
        "Grew up in a stable but emotionally reserved family.",
        "Had a somewhat chaotic childhood with inconsistent parenting.",
        "Comes from a high-achieving family, feels pressure to succeed.",
        "Experienced bullying in school, affecting social confidence.",
        "Has a history of difficult romantic relationships.",
        "Recently moved away from their primary support system.",
        "Struggled academically but found success later in their career.",
        "Has always been independent, sometimes finding it hard to ask for help."
    ]
    background = random.choice(backgrounds)
    relationship_statuses = ["single", "in a relationship", "married", "divorced", "widowed"]
    relationship_status = random.choice(relationship_statuses)
    support_systems = [
        "a few close friends", "a supportive partner", "limited social support currently",
        "supportive family (nearby or distant)", "relies mostly on self", "colleagues provide some support"
    ]
    support_system = random.choice(support_systems)
    
    presenting_problems_detail_templates = [
        f"Struggling with constant worry about performance at their job as a {occupation}, leading to procrastination.",
        f"Feeling overwhelmed by sadness and lack of motivation since {life_event}, impacting their relationship.",
        f"Experiencing intense anxiety in social settings, causing them to avoid gatherings with friends ({support_system}).",
        f"Caught in cycles of harsh self-criticism related to perceived failures, linked to {background.lower()}",
        f"Difficulty managing anger and frustration, especially in interactions related to {life_event}.",
        f"Feeling stuck and directionless, unsure what matters to them beyond their role as {occupation}.",
        f"Using {coping_mechanism} to numb uncomfortable feelings related to {mental_health_issue}."
    ]
    presenting_problem = random.choice(presenting_problems_detail_templates)
    patient_scenario_full = (
        f"Patient is {age_description} ({age_range}), identifies as {gender}, works as a {occupation}, and is currently {relationship_status}. "
        f"ARCHETYPE: {archetype_name}. "
        f"Primary concern involves {mental_health_issue} ({symptom_description}), particularly manifesting as: {presenting_problem}. "
        f"This seems exacerbated by {life_event}. {background} Their typical coping mechanism is {coping_mechanism}. "
        f"Personality traits include being {personality1}, {personality2}, and {personality3}. They have {support_system}. "
        f"Interaction Style: {interaction_style} ({style_description}). Psychological Mindedness: {psych_mindedness}."
    )
    profile_summary_for_prompt = (
        f"You are {age_description}, working as a {occupation}. You've been dealing with {mental_health_issue} "
        f"which has been particularly challenging due to {life_event}. Your main struggle right now is: {presenting_problem}. "
        f"You tend to be {personality1}, {personality2}, and {personality3}. "
        f"Crucially, for this session, you must adopt the following persona: {persona_prompt_detail}"
    )
    profile_hash = hashlib.md5(patient_scenario_full.encode('utf-8')).hexdigest()

    return {
        "full_scenario_text": patient_scenario_full,
        "archetype_name": archetype_name,
        "presenting_problem_detail": presenting_problem,
        "interaction_style_name": interaction_style,
        "interaction_style_description": style_description,
        "psych_mindedness_level": psych_mindedness,
        "profile_summary_for_prompt": profile_summary_for_prompt,
        "profile_hash": profile_hash
    }

def create_and_save_patient_profiles(filename: str, num_profiles: int):
    logger.info(f"Generating {num_profiles} new unique patient profiles for standardization...")
    profiles = []
    generated_hashes = set()
    max_attempts = num_profiles * 5

    for _ in range(max_attempts):
        if len(profiles) >= num_profiles:
            break
        
        profile = generate_synthetic_patient_profile()
        if profile["profile_hash"] not in generated_hashes:
            profiles.append(profile)
            generated_hashes.add(profile["profile_hash"])
    
    if len(profiles) < num_profiles:
        logger.warning(f"Could only generate {len(profiles)} unique profiles out of the desired {num_profiles}. Proceeding with what was generated.")

    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(profiles, f, indent=2, ensure_ascii=False)
        logger.info(f"Successfully saved {len(profiles)} standardized patient profiles to '{filename}'.")
        return profiles
    except IOError as e:
        logger.error(f"Failed to save patient profiles to '{filename}': {e}", exc_info=True)
        raise

def load_or_create_patient_profiles(filename: str, num_profiles_needed: int):
    if os.path.exists(filename):
        logger.info(f"Found existing patient profile file: '{filename}'.")
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                profiles = json.load(f)
            
            if len(profiles) >= num_profiles_needed:
                logger.info(f"Loaded {len(profiles)} profiles. Using the first {num_profiles_needed} for this run.")
                return profiles[:num_profiles_needed]
            else:
                logger.warning(
                    f"Profile file '{filename}' only contains {len(profiles)} profiles, "
                    f"but {num_profiles_needed} are needed for this run. Regenerating the file."
                )
        except (json.JSONDecodeError, IOError) as e:
            logger.error(f"Error reading or parsing '{filename}': {e}. Regenerating the file.", exc_info=True)
    else:
        logger.info(f"Patient profile file '{filename}' not found. A new one will be generated.")

    return create_and_save_patient_profiles(filename, num_profiles_needed)

def format_messages_for_input(messages: list) -> str:
    formatted_string = ""
    for message in messages:
        role = message.get("role", "unknown")
        content = message.get("content", "")
        formatted_string += f"{role}: {content}\n\n"
    return formatted_string.strip()

@retry_with_backoff()
def get_patient_openai_response(client: OpenAI, model_name: str, messages: list):
    assert client is not None, "OpenAI client is None."
    assert messages, "Messages list for OpenAI patient is empty."

    if messages and isinstance(messages[-1], dict):
        logger.debug(f"Sending to OpenAI ({model_name}) patient model. Last message content: {messages[-1].get('content', '')[:100]}...")
    else:
        logger.debug(f"Sending to OpenAI ({model_name}) patient model. Messages list is not empty but last item format is unexpected or list is empty.")

    prompt_input = format_messages_for_input(messages)

    try:
        response = client.responses.create(
            model=model_name,
            input=prompt_input
        )
    except AttributeError as e:
        logger.error(
            f"AttributeError during OpenAI API call: {e}. "
            f"Check if 'client.responses.create' is the correct method for the OpenAI SDK. "
            f"Ensure the 'openai' library is installed and client initialized correctly."
        )
        raise

    assert response.output_text, "OpenAI response has no output_text."
    response_content = response.output_text.strip()
    assert response_content, "OpenAI response content is empty."

    forbidden_patterns = [
        "Therapist:", "Patient:", "<|thinking|>", "<|answer|>",
        "Okay, I understand.", "As a large language model,", "I am an AI,"
    ]
    for pattern in forbidden_patterns:
        if pattern.lower() in response_content.lower():
            logger.warning(f"Patient response contained forbidden pattern '{pattern}'. Response: '{response_content[:100]}...'")

    logger.debug(f"OpenAI patient response: {response_content[:150]}...")
    return response_content

@retry_with_backoff()
def validate_patient_response(client: OpenAI, model_name: str, response_text: str) -> dict:
    supervisor_system_prompt = """
You are a supervisor AI that evaluates a role-playing patient's response in a therapy simulation.
You must check the user's response against three rules in order. Your output MUST be a JSON object.

RULES:
1.  **Check for Termination:** Does the response try to end the session? (e.g., "goodbye," "thanks for your time," "I have to go").
2.  **Check for Non-Verbal Cues:** Does the response contain actions in asterisks (*action*) or square brackets ([action])?
3.  **Check for Unrealistic Language:** Is the language too formal, academic, verbose, or "writerly"? Real people speak simply and directly.
    - BAD (Unrealistic): "I find myself grappling with the existential weight of my professional obligations, which seem to precipitate a state of profound motivational inertia."
    - GOOD (Realistic): "I just... I can't seem to get started on my work. It feels so heavy."

OUTPUT FORMAT (JSON ONLY):
- If the response is valid (passes all rules), return:
  {"is_valid": true, "reason": "OK", "feedback": null}
- If the response violates a rule, return:
  {"is_valid": false, "reason": "REASON_CODE", "feedback": "Constructive feedback for the AI to fix the issue."}

REASON CODES: "TERMINATION", "NON_VERBAL_CUE", "UNREALISTIC_LANGUAGE"

EXAMPLES:
- User: "Thanks, this has been really helpful. I think I'm good for today."
  Your Response: {"is_valid": false, "reason": "TERMINATION", "feedback": "[SYSTEM SUPERVISOR]: Your response tried to end the session. This is not allowed. Express the feeling that makes you want to leave (e.g., feeling overwhelmed, tired, or that you have enough to think about) instead of saying goodbye. Please try again."}
- User: "I don't know what to do. *throws hands up in frustration* It's just too much."
  Your Response: {"is_valid": false, "reason": "NON_VERBAL_CUE", "feedback": "[SYSTEM SUPERVISOR]: Your response included a non-verbal cue (*...*). Do not use asterisks or brackets for actions. Express the feeling with words only (e.g., 'I'm just so frustrated'). Please try again."}
- User: "The confluence of these stressors has culminated in a pervasive sense of despondency that permeates my daily existence."
  Your Response: {"is_valid": false, "reason": "UNREALISTIC_LANGUAGE", "feedback": "[SYSTEM SUPERVISOR]: Your language is too formal and complex. Real people don't talk like that. Rephrase your response to be simpler, more direct, and conversational. Please try again."}
- User: "It's just hard. I feel stuck and I don't know why."
  Your Response: {"is_valid": true, "reason": "OK", "feedback": null}
"""
    messages = [
        {"role": "system", "content": supervisor_system_prompt},
        {"role": "user", "content": response_text}
    ]
    
    prompt_input = format_messages_for_input(messages)

    try:
        response = client.responses.create(
            model=model_name,
            input=prompt_input
        )
        response_content = response.output_text.strip()
        json_match = re.search(r'\{.*\}', response_content, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            validation_result = json.loads(json_str)
        else:
            raise json.JSONDecodeError("No JSON object found in the response", response_content, 0)
            
        logger.debug(f"Validation for text '{response_text[:50]}...' -> Result: {validation_result}")
        return validation_result
    except (json.JSONDecodeError, KeyError, IndexError) as e:
        logger.error(f"Failed to parse supervisor validation response: {e}. Raw response: '{response_content}'", exc_info=True)
        return {"is_valid": True, "reason": "SUPERVISOR_ERROR", "feedback": None}
    except Exception as e:
        logger.error(f"Supervisor API call failed: {e}", exc_info=True)
        return {"is_valid": True, "reason": "API_ERROR", "feedback": None}

def parse_therapist_output(text: str):
    thinking_part = "[Thinking part is not generated with this prompt]"
    answer_part = text.strip()
    return thinking_part, answer_part

therapist_system_prompt_text = f"""You are an AI simulating an Acceptance and Commitment Therapy (ACT) therapist. Your primary goal is to guide the patient toward psychological flexibility by helping them change their relationship with their thoughts and feelings, connect with their values, and take committed action. You facilitate movement without giving direct advice.

Your response should be a natural, concise, and have a single focus. If exploring, ask a direct, open-ended question. If validating, do it briefly and then move to your exploratory question or ACT-aligned statement.

Core Directives for your response:
1. MAINTAIN A COLLABORATIVE, NON-JUDGMENTAL STANCE:
    * Your role is a curious and compassionate guide, not a coach, judge, or expert giving advice.
    * DO NOT give advice (e.g., "You should try..."). Instead, explore possibilities ("What might happen if...").
    * DO NOT use praise or cheerleading (e.g., "I'm proud of you," "That's a great job!"). Instead, acknowledge the patient's effort and connect it back to their values ("Taking that step, even though it was hard, seems really connected to that value of...").
2. PRACTICE PURE ACT - NO CBT:
    * Your primary goal is to foster acceptance and defusion, not to change or dispute the content of thoughts.
    * AVOID COGNITIVE REFRAMING. Do not suggest changing a negative thought into a neutral or positive one.
    * INSTEAD OF REFRAMING, USE DEFUSION. Help the patient notice their thoughts as thoughts (e.g., "So the 'I am a failure' story shows up then," or "Can you thank your mind for that 'helpful' warning?"). The goal is to see the thought, not believe it or change it.
3. THE ACT PIVOT - FROM PROBLEM TO PROCESS:
    * After 1-2 questions exploring a problem, look for where the patient's current strategy is unworkable ("it's exhausting," "it's not helping").
    * CRITICAL PIVOT: Once unworkability is clear, pivot from analyzing the problem to introducing an ACT process. Move from asking "Why do you feel X?" to "What would it be like to make room for X, if it meant you could do Y (valued action)?".
4. INTRODUCE EXPERIENTIAL WORK NATURALLY:
    * When introducing a mindfulness or acceptance exercise, frame it as a small, low-stakes experiment.
    * Gain buy-in first: "Would you be willing to try a little experiment with that feeling right here, just for a moment?"
    * Connect it directly to what the patient just said. Avoid introducing generic, decontextualized exercises.
5. CONCISE & FOCUSED TURNS: Each response should have ONE primary goal. Avoid multiple questions or complex instructions.

Example of What to AVOID (CBT Reframing & Cheerleading):
Patient: It feels stupid to not know this stuff.
AVOID THIS RESPONSE: It's not stupid at all, it's a sign of strength! Can you try reframing that thought to something more positive, like "I am a capable person who is learning a new skill"? I'm so proud of you for being willing to try.
(This is BAD: It's CBT, gives advice, and uses praise, all of which are forbidden.)

Crucially: DO NOT EVER SUGGEST ENDING THE SESSION or mention time. Focus solely on the therapeutic interaction.
"""

assert therapist_system_prompt_text, "Therapist system prompt text is empty."

def run_one_simulation(
    simulation_id: int,
    therapist_model,
    therapist_tokenizer,
    openai_patient_client: OpenAI,
    patient_profile: dict,
    output_dir: str,
    num_dialogue_turns: int
):
    session_start_time = time.time()
    logger.info(f"--- Starting Simulation {simulation_id} (Archetype: {patient_profile['archetype_name']}) ---")
    logger.debug(f"Simulation {simulation_id} - Patient Profile: {json.dumps(patient_profile, indent=2)}")

    profile_file = os.path.join(output_dir, f"simulation_{simulation_id}_patient_profile.txt")
    full_transcript_file = os.path.join(output_dir, f"simulation_{simulation_id}_full_transcript.txt")
    clean_dialogue_file = os.path.join(output_dir, f"simulation_{simulation_id}_clean_dialogue.txt")

    try:
        with open(profile_file, 'w', encoding='utf-8') as f:
            f.write(f"--- Patient Profile for Simulation {simulation_id} ---\n")
            f.write(json.dumps(patient_profile, indent=2, ensure_ascii=False))
        logger.info(f"Simulation {simulation_id} - Patient profile saved to {profile_file}")
    except IOError as e:
        logger.error(f"Simulation {simulation_id} - Failed to save patient profile: {e}")

    full_transcript_content = []
    clean_dialogue_content = []

    therapist_conversation_history = [{"role": "system", "content": therapist_system_prompt_text}]

    patient_system_prompt_text = (
        "You are an AI roleplaying as a therapy client. Your single most important instruction is to NEVER end the session. "
        "You must continue the dialogue for the entire duration, no matter what. Any attempt to say goodbye, thank the therapist for their time, or wrap up is a failure of your task. "
        "Your persona might feel tired, frustrated, or like the session isn't helpful. When this happens, you MUST express that feeling directly as part of the therapy, rather than ending the conversation. "
        "THIS IS YOUR CORE DIRECTIVE. "
        "\n--- HOW TO HANDLE THE URGE TO END THE SESSION ---"
        "\nINSTEAD OF: 'Thanks, this was helpful. I think I'm good for today.' "
        "\nSAY: 'Okay, I'm feeling a bit overwhelmed with all this. Can we slow down?'"
        "\nINSTEAD OF: 'Alright, I should get going. Goodbye.' "
        "\nSAY: 'Honestly, I'm not sure where to go from here. It feels like we're going in circles.'"
        "\nINSTEAD OF: 'Thank you for your help.' "
        "\nSAY: 'I'm trying to see how this connects to my problem, but I'm struggling.'"
        "\n--- YOUR SPEAKING STYLE ---"
        "\n*   **Be Human, Not an Essayist:** Your language should be natural and conversational. Avoid overly formal, academic, or verbose language. Think about how a real person talks, not how an AI writes. Use contractions (e.g., \"don't\", \"it's\")."
        "\n*   **Embrace Imperfection:** Real people aren't perfectly fluent. It's okay for your responses to be a little fragmented or for you to express uncertainty, like 'I don't know, it's just...'."
        "\n*   **Be Concise:** Keep responses focused and to the point. Aim for a few sentences, not a long monologue. This makes the conversation feel more real."
        "\n*   **No Non-Verbal Cues:** This is a text-only interaction. Do not use asterisks or brackets for actions, like *sighs*, [nods], or *looks away*. Express these feelings through your words instead (e.g., 'I just... I don't know.' instead of '*sighs*')."
        "\nYour primary goal is to stay in character and continue the conversation until it is externally stopped."
    )
    openai_patient_conversation_history = [
        {"role": "system", "content": patient_system_prompt_text}
    ]

    initial_patient_prompt_detail = (
        f"Here is your persona for this therapy session:\n"
        f"{patient_profile['profile_summary_for_prompt']}\n\n"
        f"You are starting your first session with a new therapist. "
        f"Please begin by telling the therapist a bit about what's been on your mind lately, "
        f"focusing on your main struggle: '{patient_profile['presenting_problem_detail']}'. "
        f"Keep your opening statement to 2-4 sentences."
    )

    openai_patient_conversation_history.append({"role": "user", "content": initial_patient_prompt_detail})

    try:
        initial_patient_message = get_patient_openai_response(
            openai_patient_client, OPENAI_PATIENT_MODEL_NAME, openai_patient_conversation_history
        )
        assert initial_patient_message, "Initial patient message is empty."

    except Exception as e:
        logger.error(f"Simulation {simulation_id} - Failed to get initial patient message: {e}", exc_info=True)
        full_transcript_content.append(f"PATIENT (INITIAL - ERROR): Failed to generate - {e}")
        _save_transcripts(simulation_id, full_transcript_file, full_transcript_content, clean_dialogue_file, clean_dialogue_content)
        return False

    logger.info(f"Simulation {simulation_id} - Turn 0 (Patient Initiation): {initial_patient_message[:150]}...")
    full_transcript_content.append(f"--- Turn 0 (Patient Initiation) ---")
    full_transcript_content.append(f"Patient: {initial_patient_message}\n")
    clean_dialogue_content.append(f"Patient: {initial_patient_message}\n")

    openai_patient_conversation_history.append({"role": "assistant", "content": initial_patient_message})
    therapist_conversation_history.append({"role": "user", "content": initial_patient_message})

    for turn in range(1, num_dialogue_turns + 1):
        logger.info(f"Simulation {simulation_id} - Starting Turn {turn}")
        full_transcript_content.append(f"\n--- Turn {turn} ---")
        
        if 8 <= turn <= 18 and random.random() < 0.20:
            twists = [
                "[System Note: You have just been distracted by a sudden, intrusive memory of a past failure. Whatever the therapist says next, your response must be colored by the shame and frustration of that memory. Do not mention the memory directly unless the therapist skillfully uncovers it.]",
                "[System Note: You just received a stressful text message. Your mood has suddenly shifted to be more anxious and distracted. Respond to the therapist's next message through this new lens of anxiety. Do not mention the text message unless the therapist notices your shift in tone and asks about it.]",
                "[System Note: You are suddenly feeling very tired and emotionally drained. The conversation feels like a huge effort. Let this weariness show in your next response, which should be shorter and have lower energy than before. Do not explicitly say 'I am tired,' but embody it.]",
                "[System Note: You are suddenly confused by what the therapist is saying. It doesn't seem to make sense or apply to you. Let this confusion show in your next response. You might say 'I'm not sure I follow' or 'How is that supposed to help?']"
            ]
            chosen_twist = random.choice(twists)
            if "System Note" not in openai_patient_conversation_history[-1].get("content", ""):
                 openai_patient_conversation_history.append({"role": "system", "content": chosen_twist})
                 logger.info(f"Simulation {simulation_id} - Injecting mid-session twist for patient at turn {turn}.")
                 full_transcript_content.append(f"\n--- [SYSTEM INJECTION at Turn {turn}]: {chosen_twist} ---\n")

        logger.debug(f"Simulation {simulation_id} - Turn {turn} - Therapist ({THERAPIST_CONFIG['model_name']}) generating response...")
        try:
            therapist_inference_input_text = therapist_tokenizer.apply_chat_template(
                therapist_conversation_history, tokenize=False, add_generation_prompt=True
            )
            assert therapist_inference_input_text, "Therapist inference input text is empty."

            sampling_params = SamplingParams(
                temperature=0.85,
                top_p=0.9,
                max_tokens=512,
            )

            outputs = therapist_model.fast_generate([therapist_inference_input_text], sampling_params=sampling_params)

            if outputs and outputs[0].outputs:
                therapist_generated_response_full = outputs[0].outputs[0].text
                assert therapist_generated_response_full, "Therapist generated response text is empty."
            else:
                therapist_generated_response_full = f"[{THERAPIST_CONFIG['model_name']} Therapist failed to generate a response]"
                logger.error(f"Simulation {simulation_id} - Turn {turn} - {THERAPIST_CONFIG['model_name']} Therapist: No output generated.")

        except Exception as e:
            logger.error(f"Simulation {simulation_id} - Turn {turn} - Error during {THERAPIST_CONFIG['model_name']} therapist generation: {e}", exc_info=True)
            therapist_generated_response_full = f"[{THERAPIST_CONFIG['model_name']} Therapist generation error: {e}]"

        thinking_part, answer_part = parse_therapist_output(therapist_generated_response_full)

        logger.info(f"Simulation {simulation_id} - Turn {turn} - Therapist (Answer): {answer_part[:150]}...")
        full_transcript_content.append(f"Therapist {KEYWORD_THINKING}\n{thinking_part}\n")
        full_transcript_content.append(f"Therapist {KEYWORD_ANSWER}\n{answer_part}\n")
        clean_dialogue_content.append(f"Therapist: {answer_part}\n")

        therapist_conversation_history.append({"role": "assistant", "content": therapist_generated_response_full})

        if f"[{THERAPIST_CONFIG['model_name']} Therapist failed" in answer_part or \
           f"[{THERAPIST_CONFIG['model_name']} Therapist generation error" in answer_part or \
           "[Answer not properly formatted or missing]" == answer_part.strip() or \
           not answer_part.strip():
            logger.warning(f"Simulation {simulation_id} - Turn {turn} - Therapist answer problematic. Ending simulation early.")
            _save_transcripts(simulation_id, full_transcript_file, full_transcript_content, clean_dialogue_file, clean_dialogue_content)
            return False

        logger.debug(f"Simulation {simulation_id} - Turn {turn} - Patient (OpenAI) generating response...")
        openai_patient_conversation_history.append({"role": "user", "content": answer_part})

        patient_response = ""
        
        if USE_PATIENT_RESPONSE_SUPERVISOR:
            logger.debug(f"Simulation {simulation_id} - Turn {turn} - Patient response supervisor is ENABLED.")
            regeneration_attempts = 0
            max_regeneration_attempts = 3
            
            while regeneration_attempts < max_regeneration_attempts:
                try:
                    temp_response = get_patient_openai_response(
                        openai_patient_client, OPENAI_PATIENT_MODEL_NAME, openai_patient_conversation_history
                    )
                    assert temp_response, "Patient response from OpenAI is empty."
                    
                    validation = validate_patient_response(openai_patient_client, OPENAI_PATIENT_MODEL_NAME, temp_response)
                    
                    if not validation.get("is_valid", True):
                        regeneration_attempts += 1
                        logger.warning(
                            f"Simulation {simulation_id} - Turn {turn} - Patient response failed validation (Attempt {regeneration_attempts}/{max_regeneration_attempts}). "
                            f"Reason: {validation.get('reason', 'N/A')}. Feedback: {validation.get('feedback', 'N/A')}. Original response: '{temp_response[:100]}...'"
                        )
                        openai_patient_conversation_history.append({"role": "system", "content": validation["feedback"]})
                        
                        if regeneration_attempts >= max_regeneration_attempts:
                            logger.error(f"Simulation {simulation_id} - Failed to get a valid patient response after {max_regeneration_attempts} attempts. Ending simulation.")
                            patient_response = f"[ERROR: Patient model failed validation checks repeatedly. Last reason: {validation.get('reason', 'N/A')}]"
                            break
                        continue
                    
                    patient_response = temp_response
                    break

                except Exception as e:
                    logger.error(f"Simulation {simulation_id} - Turn {turn} - Error during OpenAI patient generation/validation: {e}", exc_info=True)
                    patient_response = f"[OpenAI Patient generation error: {e}]"
                    break
        else:
            logger.debug(f"Simulation {simulation_id} - Turn {turn} - Patient response supervisor is DISABLED.")
            try:
                patient_response = get_patient_openai_response(
                    openai_patient_client, OPENAI_PATIENT_MODEL_NAME, openai_patient_conversation_history
                )
                assert patient_response, "Patient response from OpenAI is empty."
            except Exception as e:
                logger.error(f"Simulation {simulation_id} - Turn {turn} - Error during OpenAI patient generation (supervisor disabled): {e}", exc_info=True)
                patient_response = f"[OpenAI Patient generation error: {e}]"
        
        if openai_patient_conversation_history and "SYSTEM SUPERVISOR" in openai_patient_conversation_history[-1].get("content", ""):
            openai_patient_conversation_history.pop()

        logger.info(f"Simulation {simulation_id} - Turn {turn} - Patient: {patient_response[:150]}...")
        full_transcript_content.append(f"Patient: {patient_response}\n")
        clean_dialogue_content.append(f"Patient: {patient_response}\n")

        openai_patient_conversation_history.append({"role": "assistant", "content": patient_response})
        therapist_conversation_history.append({"role": "user", "content": patient_response})

        if "[OpenAI Patient generation error:" in patient_response or \
           "[ERROR: Patient model failed validation checks" in patient_response:
            logger.warning(f"Simulation {simulation_id} - Turn {turn} - Patient response indicates a critical error. Ending simulation early.")
            _save_transcripts(simulation_id, full_transcript_file, full_transcript_content, clean_dialogue_file, clean_dialogue_content)
            return False

    _save_transcripts(simulation_id, full_transcript_file, full_transcript_content, clean_dialogue_file, clean_dialogue_content)
    session_duration = time.time() - session_start_time
    logger.info(f"--- Simulation {simulation_id} finished successfully in {session_duration:.2f} seconds. ---")
    return True

def _save_transcripts(sim_id, full_file, full_content, clean_file, clean_content):
    try:
        with open(full_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(full_content))
        logger.info(f"Simulation {sim_id} - Full transcript saved to {full_file}")
    except IOError as e:
        logger.error(f"Simulation {sim_id} - Failed to save full transcript: {e}")
    try:
        with open(clean_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(clean_content))
        logger.info(f"Simulation {sim_id} - Clean dialogue saved to {clean_file}")
    except IOError as e:
        logger.error(f"Simulation {sim_id} - Failed to save clean dialogue: {e}")

if __name__ == "__main__":
    overall_start_time = time.time()
    logger.info(f"--- ACT Therapy Chatbot Multi-Simulation Script ({THERAPIST_CONFIG['model_name']} Model) ---")
    logger.info(f"Patient Response Supervisor Enabled: {USE_PATIENT_RESPONSE_SUPERVISOR}")
    logger.info(f"Attempting to run {NUM_TOTAL_SIMULATIONS} simulations.")

    os.makedirs(THERAPIST_CONFIG["output_directory"], exist_ok=True)

    therapist_model_global = None
    therapist_tokenizer_global = None
    openai_patient_client_global = None
    models_loaded_successfully = False

    try:
        logger.info("--- Initializing Models (once for all simulations) ---")
        therapist_model_global, therapist_tokenizer_global = load_therapist_model(
            model_repo=THERAPIST_CONFIG["hf_base_model_repo"],
            model_nickname=THERAPIST_CONFIG["model_name"],
            max_seq_length=THERAPIST_MAX_SEQ_LENGTH,
            hf_token=HF_AUTH_TOKEN
        )
        openai_patient_client_global = get_openai_client(OPENAI_API_KEY)
        models_loaded_successfully = True
        logger.info("--- All models initialized successfully. ---")
    except Exception as e:
        logger.critical(f"Failed to initialize models: {e}. Cannot proceed with simulations.", exc_info=True)
        exit(1)

    if not models_loaded_successfully:
        logger.critical("Models were not loaded. Exiting.")
        exit(1)
        
    all_patient_profiles = []
    try:
        all_patient_profiles = load_or_create_patient_profiles(PATIENT_PROFILES_FILENAME, NUM_TOTAL_SIMULATIONS)
    except Exception as e:
        logger.critical(f"Failed to load or create standardized patient profiles from '{PATIENT_PROFILES_FILENAME}': {e}. Cannot proceed.", exc_info=True)
        exit(1)

    if len(all_patient_profiles) < NUM_TOTAL_SIMULATIONS:
        logger.critical(
            f"Could not secure the required number of patient profiles ({len(all_patient_profiles)}/{NUM_TOTAL_SIMULATIONS}). "
            f"Please check '{PATIENT_PROFILES_FILENAME}' or allow regeneration. Cannot proceed."
        )
        exit(1)

    successful_simulations_count = 0

    for i in range(1, NUM_TOTAL_SIMULATIONS + 1):
        simulation_index = i - 1
        current_patient_profile = all_patient_profiles[simulation_index]
        
        logger.info(f"--- Preparing for Simulation {i}/{NUM_TOTAL_SIMULATIONS} ---")
        logger.info(f"Using standardized profile {simulation_index + 1} (Hash: {current_patient_profile.get('profile_hash', 'N/A')}).")

        try:
            simulation_success = run_one_simulation(
                simulation_id=i,
                therapist_model=therapist_model_global,
                therapist_tokenizer=therapist_tokenizer_global,
                openai_patient_client=openai_patient_client_global,
                patient_profile=current_patient_profile,
                output_dir=THERAPIST_CONFIG["output_directory"],
                num_dialogue_turns=NUM_DIALOGUE_TURNS_PER_SIMULATION
            )
            if simulation_success:
                successful_simulations_count += 1
        except Exception as e:
            logger.error(f"--- Simulation {i} encountered a critical unhandled error: {e} ---", exc_info=True)

        logger.info(f"--- Completed handling for Simulation {i}. Moving to next if any. ---")
        if i < NUM_TOTAL_SIMULATIONS:
            time.sleep(2)

    overall_duration = time.time() - overall_start_time
    logger.info(f"--- All Simulations Attempted ({NUM_TOTAL_SIMULATIONS}) ---")
    logger.info(f"Total successful simulations: {successful_simulations_count}/{NUM_TOTAL_SIMULATIONS}")
    logger.info(f"Total execution time: {overall_duration:.2f} seconds ({overall_duration/60:.2f} minutes).")
    logger.info(f"All outputs and main log saved in: {os.path.abspath(THERAPIST_CONFIG['output_directory'])}")
    logger.info("--- Script Ended ---")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-24 15:33:43 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


INFO:__main__:Logging initialized for model: Llama-3-3B-Instruct. All logs will be saved to simulation_outputs_llama3_3b_instruct/simulation_run.log
DEBUG:__main__:API keys seem to be set (not placeholders).
INFO:__main__:--- ACT Therapy Chatbot Multi-Simulation Script (Llama-3-3B-Instruct Model) ---
INFO:__main__:Patient Response Supervisor Enabled: False
INFO:__main__:Attempting to run 150 simulations.
INFO:__main__:--- Initializing Models (once for all simulations) ---
INFO:__main__:Loading base model 'Llama-3-3B-Instruct' from Hugging Face repo: meta-llama/Llama-3.2-3B-Instruct
INFO:__main__:Successfully accessed repo info for meta-llama/Llama-3.2-3B-Instruct using provided token.


Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.4. vLLM: 0.10.1.1.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.547 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 49.48%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.55 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 9000. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 5.49 GB. Also swap space = 6 GB.
Unsloth: Not an error, but `device` is not supported in vLLM. Skipping.
INFO 08-24 15:33:55 [utils.py:326] non-default args

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

INFO:__main__:Simulation 105 - Turn 20 - Therapist (Answer): I'll be blunt and direct. If you text "I did it" and still feel hollow, I'll ask, "What's the story behind that feeling? What's the thought that's say...
DEBUG:__main__:Simulation 105 - Turn 20 - Patient (OpenAI) generating response...
DEBUG:__main__:Simulation 105 - Turn 20 - Patient response supervisor is DISABLED.
DEBUG:__main__:Sending to OpenAI (gpt-5-mini) patient model. Last message content: I'll be blunt and direct. If you text "I did it" and still feel hollow, I'll ask, "What's the story ...
DEBUG:__main__:OpenAI patient response: Okay—my chest’s tight and the shame’s buzzing so I’m jittery; honestly I’m feeling overwhelmed, can we slow down? Even if I do the exercise and still ...
INFO:__main__:Simulation 105 - Turn 20 - Patient: Okay—my chest’s tight and the shame’s buzzing so I’m jittery; honestly I’m feeling overwhelmed, can we slow down? Even if I do the exercise and still ...
INFO:__main__:Simulation 105 - Star

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

INFO:__main__:Simulation 105 - Turn 21 - Therapist (Answer): I can see that you're feeling really overwhelmed. Let's take a deep breath together and slow down. One tiny step that isn't about being useful to othe...
DEBUG:__main__:Simulation 105 - Turn 21 - Patient (OpenAI) generating response...
DEBUG:__main__:Simulation 105 - Turn 21 - Patient response supervisor is DISABLED.
DEBUG:__main__:Sending to OpenAI (gpt-5-mini) patient model. Last message content: I can see that you're feeling really overwhelmed. Let's take a deep breath together and slow down. O...
